In [2]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import numpy as np
import skfuzzy as fuzz
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
import graphviz
import os
#Path will change depending on where graphviz executables are located
#If path is already established in environment variables, it may not be needed
# os.environ["PATH"] += os.pathsep + 'C:/Program Files/Graphviz/bin'

In [3]:
c0 = ["hot", "hot", "hot", "mild", "cool", "cool", "cool", "mild", "cool", "mild", "mild", "mild", "hot",
      "mild"]
c1 = ["weak", "strong", "weak", "weak", "weak", "strong", "strong", "weak", "weak", "weak", "strong", "strong",
      "weak", "strong"]
c2 = ["long", "long", "long", "long", "short", "short", "short", "long", "short", "short", "short", "long",
      "short", "long"]
c3 = ["no", "no", "yes", "yes", "yes", "no", "yes", "no", "yes", "yes", "yes", "yes", "yes", "no"]

data_frame = pd.DataFrame([c0, c1, c2, c3]).T

In [4]:
#Using label encoder to numerize categories
label_encoder = LabelEncoder()
data_frame[0] = label_encoder.fit_transform(data_frame[0])
data_frame[1] = label_encoder.fit_transform(data_frame[1])
data_frame[2] = label_encoder.fit_transform(data_frame[2])
data_frame[3] = label_encoder.fit_transform(data_frame[3])
y = data_frame.iloc[:, 3]

In [5]:
# Converting to numpy to use skfuzzy
temp_numpy = np.asarray(data_frame[0])
wind_numpy = np.asarray(data_frame[1])
traffic_numpy = np.asarray(data_frame[2])
temp_fuzzy = fuzz.trimf(temp_numpy, [0, 1.25, 2])
wind_fuzzy = fuzz.trimf(wind_numpy, [0, 0.01, 1])
traffic_fuzzy = fuzz.trimf(traffic_numpy, [0, 0.999, 1])

X = pd.DataFrame([temp_fuzzy, wind_fuzzy, traffic_fuzzy]).T

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=0.1,
                                                    random_state=42)

clf = DecisionTreeClassifier(random_state=0,
                             criterion="entropy")
clf.fit(X_train, y_train)

DecisionTreeClassifier(criterion='entropy', random_state=0)

In [7]:
features = ['temperature', 'wind', 'traffic']
response = ['no', 'yes']

dot_data = tree.export_graphviz(clf,
                                out_file=None,
                                feature_names=features,
                                class_names=response,
                                filled=True)
# Draw graph
graph = graphviz.Source(dot_data, format="png")
graph
graph.render("car_driving_tree_fuzzy")
print("Decision Fuzzy Tree score: ", clf.score(X_test, y_test))

Decision Fuzzy Tree score:  1.0
